# Классификация комментариев

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

В нашем распоряжении набор данных с разметкой о токсичности правок.

**Цель исследования**

Предсказать токсичные комментарии пользователей.

**Задача исследования**

Обучить модель классифицировать комментарии на позитивные и негативные. 

Построить модель со значением метрики качества *F1* не меньше 0.75.

**Описание данных**

В CSV-файле столбец *text* содержит текст комментария, а *toxic* — целевой признак.

In [1]:
import pandas as pd
import numpy as np

import re 
import nltk
nltk.download('wordnet', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('punkt', quiet=True)
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from nltk.corpus import wordnet
from nltk.corpus import stopwords as nltk_stopwords

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
RANDOM_STATE = 42
TEST_SIZE = 0.25

from tqdm import tqdm # для progress_apply
tqdm.pandas()         # для progress_apply

## Подготовка

In [2]:
try:
    data = pd.read_csv('toxic_comments.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

In [3]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Пропущенных значений нет, типы данных указаны верно.

Займемся лемматизацией текста:

In [4]:
wnl = WordNetLemmatizer()

def penn2morphy(penntag):
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n' 

def lemmatize_sent(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    return ' '.join([wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) 
            for word, tag in pos_tag(word_tokenize(text))])

In [5]:
data['lemm_text'] = data['text'].progress_apply(lemmatize_sent)

100%|██████████| 159292/159292 [08:33<00:00, 310.05it/s]


Разобъем признаки на входные и целевой

In [6]:
X = data['lemm_text']
y = data['toxic']

Разобъем выборку на train и test

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = TEST_SIZE, random_state = RANDOM_STATE)

Избавимся от слов не имеющих смысловой нагрузки и вычислим TF-IDF

In [8]:
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

In [9]:
X_train_tfidf = count_tf_idf.fit_transform(X_train)
X_test_tfidf = count_tf_idf.transform(X_test)

## Обучение

Будем обучать две модели LogisticRegression и DecisionTreeClassifier, метрикой для проверки будем использовать F1_score

In [10]:
pipe_final = Pipeline([('models', LogisticRegression(random_state = RANDOM_STATE))])

In [11]:
param_distributions = [
    {
        'models': [LogisticRegression(random_state = RANDOM_STATE, penalty='elasticnet', solver='saga')],
        'models__l1_ratio': [0, 0.5, 1]
    },
    {
        'models': [DecisionTreeClassifier(random_state = RANDOM_STATE)],
        'models__max_depth': [2, 5],
        'models__min_samples_leaf': [2, 5]
    }
] 

randomized_search = GridSearchCV(
    pipe_final, 
    param_distributions, 
    scoring='f1', 
    n_jobs=-1,
    verbose=10
)

In [12]:
randomized_search.fit(X_train_tfidf, y_train);

Fitting 5 folds for each of 7 candidates, totalling 35 fits


In [13]:
print('Лучшая модель и её параметры:\n', randomized_search.best_estimator_) 

y_test_pred = randomized_search.predict(X_test_tfidf)
print(f'Метрика RMSE для лучшей модели: {randomized_search.best_score_}')

Лучшая модель и её параметры:
 Pipeline(steps=[('models',
                 LogisticRegression(l1_ratio=1, penalty='elasticnet',
                                    random_state=42, solver='saga'))])
Метрика RMSE для лучшей модели: 0.7586272969149723


Лучшей стала LogisticRegression с метрикой на тренировочных данных равной 0.758.

In [14]:
pd.DataFrame(randomized_search.cv_results_)[['mean_test_score', 'mean_fit_time',
                                                          'mean_score_time', 'param_models', 'params']]

,mean_test_score,mean_fit_time,mean_score_time,param_models,params
0,0.711528,2.547329,0.019502,"LogisticRegression(l1_ratio=1, penalty='elasti...","{'models': LogisticRegression(l1_ratio=1, pena..."
1,0.730228,33.745878,0.015522,"LogisticRegression(l1_ratio=1, penalty='elasti...","{'models': LogisticRegression(l1_ratio=1, pena..."
2,0.758627,32.802149,0.022174,"LogisticRegression(l1_ratio=1, penalty='elasti...","{'models': LogisticRegression(l1_ratio=1, pena..."
3,0.396193,14.503089,0.044363,DecisionTreeClassifier(random_state=42),{'models': DecisionTreeClassifier(random_state...
4,0.396193,12.905983,0.053306,DecisionTreeClassifier(random_state=42),{'models': DecisionTreeClassifier(random_state...
5,0.512147,15.787529,0.042375,DecisionTreeClassifier(random_state=42),{'models': DecisionTreeClassifier(random_state...
6,0.511292,12.643606,0.029490,DecisionTreeClassifier(random_state=42),{'models': DecisionTreeClassifier(random_state...


Модель DecisionTreeClassifier показала не очень хорошие результаты. По итогу лучшей моделью выбираем LogisticRegression. Посчитатем метрику на тестовой выборке:

In [15]:
print(f'Метрика RMSE на тестовой выборке: {f1_score(y_test, y_test_pred)}')

Метрика RMSE на тестовой выборке: 0.772386476163531


Метрика получила значение > 0.75, наша модель осуществляет хорошие прогнозы.

## Выводы

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах.

Мы лемматизировали текст, избавились от слов не имеющих смысловой нагрузки и вычислили TF-IDF, который использовали как входной признак. Целевый признаком был столбец toxic.

Мы обучили разные модели классифицировать комментарии на позитивные и негативные. Которые могут искать токсичные комментарии для отправления их на модерацию.

Лучшей моделью стала LogisticRegression с параметром l1_ratio = 1.

Построенная модель получила значение метрики F1 на тестовой выборке равное 0.77, она точно устроит заказчика.

Для своих коллег хочу посоветовать подобрать более тонко параметры для LogisticRegression, а именно рассмотреть параметр C в интервале от 5 до 15 в паре с подбором l1_ratio = [0, 1], что должно улучшить метрику, но при этом увеличит время выполнения.